In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob

import pandas as pd

In [3]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_656768/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_656768/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_656768/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_656768/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs
0,\AKROUSDT\plots\2022-03-07T074200\backtest_res...,5.5,AKROUSDT,True,17.780,0.0766,100.0,0.9507,True,19.350,0.0738,100.0,0.9633
1,\BALUSDT\plots\2022-03-07T211357\backtest_resu...,5.5,BALUSDT,True,4.607,0.0206,100.0,0.9197,True,14.440,0.0582,100.0,0.9341
2,\BTSUSDT\plots\2022-03-07T211621\backtest_resu...,5.5,BTSUSDT,True,12.640,0.0553,100.0,0.9315,True,19.740,0.0755,100.0,0.9572
3,\DEFIUSDT\plots\2022-03-07T185402\backtest_res...,5.5,DEFIUSDT,True,9.501,0.0420,100.0,0.9409,True,14.410,0.0480,100.0,0.9501
4,\DEFIUSDT\plots\2022-03-07T211821\backtest_res...,5.5,DEFIUSDT,True,9.501,0.0420,100.0,0.9409,True,14.410,0.0480,100.0,0.9501
...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,\ZECUSDT\plots\2022-02-25T232208\backtest_resu...,5.5,ZECUSDT,True,12.520,0.0599,100.0,0.9660,True,3.435,0.0173,100.0,0.9792
398,\ZENUSDT\plots\2022-03-05T072906\backtest_resu...,5.5,ZENUSDT,True,8.481,0.0372,100.0,0.9209,True,19.940,0.0882,100.0,0.8434
399,\ZILUSDT\plots\2022-03-06T080955\backtest_resu...,5.5,ZILUSDT,True,12.050,0.0533,100.0,0.9544,True,15.840,0.0657,100.0,0.9504
400,\ZRXUSDT\plots\2022-03-06T230914\backtest_resu...,5.5,ZRXUSDT,True,15.580,0.0683,100.0,0.9472,True,14.480,0.0616,100.0,0.9139


Now filter the dataframe

In [4]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = ["XEMUSDT", "FLMUSDT", "BALUSDT",
                       "BTSUSDT", "DGBUSDT", "DEFIUSDT"]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              & (big_data['long'] == True)
              & (big_data['short'] == True)
              & (big_data['adgt_%'] > 0.1)
              & (big_data['bnkrl_%'] > 95.0)
              & (big_data['bnkrs_%'] > 95.0)
              & (big_data['ebrs'] > .85)
              & (big_data['ebrl'] > .85)
              & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs,adgt_%
2,\BTSUSDT\plots\2022-03-07T211621\backtest_resu...,5.5,BTSUSDT,True,12.640,0.0553,100.0,0.9315,True,19.74,0.0755,100.0,0.9572,0.1308
5,\DGBUSDT\plots\2022-03-07T185158\backtest_resu...,5.5,DGBUSDT,True,11.750,0.0518,100.0,0.9583,True,22.52,0.0890,100.0,0.9309,0.1408
6,\DGBUSDT\plots\2022-03-07T211718\backtest_resu...,5.5,DGBUSDT,True,11.750,0.0518,100.0,0.9583,True,22.52,0.0890,100.0,0.9309,0.1408
7,\FLMUSDT\plots\2022-03-07T075107\backtest_resu...,5.5,FLMUSDT,True,7.212,0.0333,100.0,0.9352,True,22.12,0.0928,100.0,0.9539,0.1261
8,\FLMUSDT\plots\2022-03-07T211033\backtest_resu...,5.5,FLMUSDT,True,11.910,0.0467,100.0,0.9362,True,22.12,0.0928,100.0,0.9539,0.1395


In [5]:
df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
dx = df[['file','s','adgt_%','adgl_%','adgs_%','bnkrl_%', 'bnkrs_%','ebrl', 'ebrs']]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_656768/2043545472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs
2,\BTSUSDT\plots\2022-03-07T211621\backtest_resu...,BTSUSDT,0.1308,0.0553,0.0755,100.0,100.0,0.9315,0.9572
5,\DGBUSDT\plots\2022-03-07T185158\backtest_resu...,DGBUSDT,0.1408,0.0518,0.0890,100.0,100.0,0.9583,0.9309
7,\FLMUSDT\plots\2022-03-07T075107\backtest_resu...,FLMUSDT,0.1261,0.0333,0.0928,100.0,100.0,0.9352,0.9539
8,\FLMUSDT\plots\2022-03-07T211033\backtest_resu...,FLMUSDT,0.1395,0.0467,0.0928,100.0,100.0,0.9362,0.9539
